In [1]:
import os
import sys
import requests
import json

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from tqdm import tqdm

AXIS_FONT_SIZE = 16

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# print(notebook_dir)
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from data_processing import DataProcessing
from real_data_acquisition import OpenMeasuresDirector
from text_generation_models import TextGenerationModelFactory

In [2]:
pd.set_option('max_colwidth', 800)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

## Create prompt

In [3]:
prediction_properties = """a prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:

    1. <p_s>, any source entity in the sports domain.
        - Can be a person (with a name) or a sports domain person such as a sports reporter, sports analyst, sports expert, sports top executive, sports senior level person, etc), civilian.
        - Can only be an organization that is associated with the sports prediction.
    2. <p_t>, any target entity in the sports domain.
	    - Can be a person (with a name) or a sports person such as a sports reporter, sports analyst, sports expert, sports top executive, sports senior level person, etc).
        - Can only be an organization that is associated with the sports prediction.
    3. <p_d>, date or time range when <p> is expected to come to fruition or when one should observe the <p>.
        - Forecast can range from a second to anytime in the future.
        - Answers the questions: "How far to go out from today?" or "Where to stop?".
    4. <p_o>, sports prediction outcome.
        - Details relevant details such as outcome, a quantifiable metric, or slope.
        - Some example outcomes are the following: score, touchdown, goal, points, win, lose, etc.
"""

prediction_structures = """Here are how some sports predictions are structured:
    - sports template 1: <p_s> forecasts that the <p_o> at <p_t> potentially decrease in <p_d>.
    - sports template 2: On <p_d>, <p_s> speculates the <p_o> at <p_t> will likely increase.
    - sports template 3: <p_s> predicts on <p_d>, the <p_t> <p_o> may rise.
    - sports template 4: According to <p_s>, the <p_o> at <p_t> would fall in <p_d>.
    - sports template 5: In <p_d>, <p_s> envisions that <p_t> <p_o> has some probability to remain stable.
    - sports template 6: <p_t> <p_o> should stay same <p_d>, according to <p_s>. 
"""

sport_examples = """Here are some corresponding examples of sports predictions:
    - sport examples for template 1:
        1. Coach Lisa Martinez predicts that the touchdown rate at the Miami Dolphins will fall in 2020 of October.
        2. Analyst Mark Johnson forecasts that the goal average at Manchester United will stay the same in November 2025.
        3. Ryan forecasts win percentage he has for soccer will go up in 12/25/2016.
    - sport examples for template 2:
        1. On Sep 20, 2100, Coach Maria Lopez suggests that the score average at the Chicago Bulls is climbing.
        2. On 9/12/2025, Analyst David Kim anticipates the touchdown rate at the Kansas City Chiefs will likely surge.
        3. On October 8, 2123, Detravious foresees that the win probability he has for rugby is expected to trend downward.
    - sport examples for template 3:
        1. Coach Elena Ruiz predicts on 9/22/2025, the goal count at Real Madrid will climb.
        2. Analyst Marcus Lee forecasts that on Sep 30, 2055, the point average at the Golden State Warriors will be higher.
        3. George Jr. estimates that on October 15, 2035, the win ratio for games he has will disimprove.
    - sport examples for template 4:
        1. According to Coach Sarah Nguyen, the scoring average at the Dallas Mavericks is expected to dip in Sep 2021.
        2. According to Analyst Trevor Simmons, the touchdown rate at the Green Bay Packers will increase in 10/2025.
        3. According to Manchester United, the win percentage at Manchester United is projected to drop in October 2034.
    - sport examples for template 5:
        1. In 9/2025, Coach Miguel Torres envisions that the goal average at Paris Saint-Germain will hold steady.
        2. In October 2056, Analyst Fiona Bennett anticipates that the win rate at the Toronto Raptors will decrease slightly.
        3. In Sep 2086, Calvin foresees that the points per game he has in football will gradually increase.
    - sport examples for template 6:
        1. The goal count at Liverpool FC will surge in Sep 2012, according to Coach Daniel Alvarez.
        2. The win percentage at the Chicago Bears will taper off in October 2025, according to Analyst Priya Sharma.
        3. The scoring average on Arnolds footbal team will remain steady in 10/2034, according to Arnold.
"""

sport_requirements = """- Should be based on real-world sports.
    - Suppose the time when <p> was made is during any season of sports.
    - Include reports from all sports professionals, coaches, or any type of sport entity.
"""

initial_query_string = """(NFL OR nfl) AND (playoffs) AND (Super Bowl LIX)"""

In [4]:
prompt = f"""Generate a query string using boolean logic and keywords (related to sports predictions) to search a database. I define {prediction_properties} 
{prediction_structures}
{sport_examples}
These sports predictions can be found in social media data at large. My task here is to query the site to find relatable sentences (that aren't predictions) and prediction sentences). 
My initial query string: {initial_query_string}. Don't use brackets to wrap words nor to use quotation marks to wrap words. 
I need you to generate an improved (better prediction precision) query string taking into consideration the above along with {sport_requirements} \n Don't generate anything other than a new/imporved query string!
"""
prompt

'Generate a query string using boolean logic and keywords (related to sports predictions) to search a database. I define a prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:\n\n    1. <p_s>, any source entity in the sports domain.\n        - Can be a person (with a name) or a sports domain person such as a sports reporter, sports analyst, sports expert, sports top executive, sports senior level person, etc), civilian.\n        - Can only be an organization that is associated with the sports prediction.\n    2. <p_t>, any target entity in the sports domain.\n\t    - Can be a person (with a name) or a sports person such as a sports reporter, sports analyst, sports expert, sports top executive, sports senior level person, etc).\n        - Can only be an organization that is associated with the sports prediction.\n    3. <p_d>, date or time range when <p> is expected to come to fruition or when one should observe the <p>.\n        - Forecast 

## Query for data

- For query string, have user define `initial_query_string` or have any LLM in `text_generation_models.py` to generate via the prompt. Either way, the system is set up for user feedback. With this, check the query string (and url for data). If good with it type 'agree'. If not, add details. The details will append to old prompt.

In [5]:
# Configuring parameters
terms_for_query = OpenMeasuresDirector
query_string_by = 'user'
limit = 10
# NFL playoffs (The NFL playoffs for the 2024 season began on January 11, 2025, and concluded with Super Bowl LIX on February 9 at Caesars Superdome in New Orleans, Louisiana | WIKI)
since = '2025-01-25'
until = '2025-02-09' 
esquery = 'query_string' # Elasticsearch across all fields

# sites = ["tiktok_comment", "bluesky", "truth_social"]
# sites = ["truth_social"]
sites = ["bluesky", "truth_social"]
hits_per_site_dfs = []
for site in sites:
    hits_for_site_df = OpenMeasuresDirector.construct_from_dataset(query_string=initial_query_string, query_string_by=query_string_by, limit=limit, site=site, start_date=since, end_date=until, querytype=esquery)
    hits_per_site_dfs.append(hits_for_site_df)

=============================== Site: bluesky ===============================
### RESET ###
### USER SPECIFY QUERY STRINGS ###
	Query String: (<class 'str'>, '(NFL OR nfl) AND (playoffs) AND (Super Bowl LIX)')

### SET QUERY ###
	Query's URL: http://api.smat-app.com/content?term=(NFL OR nfl) AND (playoffs) AND (Super Bowl LIX)&limit=10&site=bluesky&since=2025-01-25&until=2025-02-09&querytype=query_string

### GET RAW HITS ###
	Hits: 200
Hits retrieved:
                $type                            author  \
0  app.bsky.feed.post  did:plc:zialmxrbi3qx7fpogrov6qwf   
1  app.bsky.feed.post  did:plc:4hj6eobdxzxn3d3u63rlxjwe   
2  app.bsky.feed.post  did:plc:bgt62gzl3y5ke7b45xi5vkvu   
3  app.bsky.feed.post  did:plc:iddtdjw5gujvbxbzjm45wlgb   
4  app.bsky.feed.post  did:plc:y545canthvu4hk5bg4jdwh2t   
5  app.bsky.feed.post  did:plc:pggdgcq644d5lppxzjyiledn   
6  app.bsky.feed.post  did:plc:zuczfxtiepgm6jdy6d2g5uvk   
7  app.bsky.feed.post  did:plc:z4547lu5ykj2ot3no6fwz5n4   
8  app.bsky.

In [6]:
hits_per_site_dfs

[[                $type                            author  \
  0  app.bsky.feed.post  did:plc:zialmxrbi3qx7fpogrov6qwf   
  1  app.bsky.feed.post  did:plc:4hj6eobdxzxn3d3u63rlxjwe   
  2  app.bsky.feed.post  did:plc:bgt62gzl3y5ke7b45xi5vkvu   
  3  app.bsky.feed.post  did:plc:iddtdjw5gujvbxbzjm45wlgb   
  4  app.bsky.feed.post  did:plc:y545canthvu4hk5bg4jdwh2t   
  5  app.bsky.feed.post  did:plc:pggdgcq644d5lppxzjyiledn   
  6  app.bsky.feed.post  did:plc:zuczfxtiepgm6jdy6d2g5uvk   
  7  app.bsky.feed.post  did:plc:z4547lu5ykj2ot3no6fwz5n4   
  8  app.bsky.feed.post  did:plc:j2wsd5dyynpwh3rrpsuhqfkv   
  9  app.bsky.feed.post  did:plc:i3bbgjnal63ful2tjryi3onb   
  
                                                                                                                                                                                                                                                                                                                                      

- in json i'm saving collect more metadataa - use all cols given from each site
1. time for each query using query process with Dr. Grant


In [7]:
hits_per_site_dfs

[[                $type                            author  \
  0  app.bsky.feed.post  did:plc:zialmxrbi3qx7fpogrov6qwf   
  1  app.bsky.feed.post  did:plc:4hj6eobdxzxn3d3u63rlxjwe   
  2  app.bsky.feed.post  did:plc:bgt62gzl3y5ke7b45xi5vkvu   
  3  app.bsky.feed.post  did:plc:iddtdjw5gujvbxbzjm45wlgb   
  4  app.bsky.feed.post  did:plc:y545canthvu4hk5bg4jdwh2t   
  5  app.bsky.feed.post  did:plc:pggdgcq644d5lppxzjyiledn   
  6  app.bsky.feed.post  did:plc:zuczfxtiepgm6jdy6d2g5uvk   
  7  app.bsky.feed.post  did:plc:z4547lu5ykj2ot3no6fwz5n4   
  8  app.bsky.feed.post  did:plc:j2wsd5dyynpwh3rrpsuhqfkv   
  9  app.bsky.feed.post  did:plc:i3bbgjnal63ful2tjryi3onb   
  
                                                                                                                                                                                                                                                                                                                                      

In [8]:
# tiktok_dfs = hits_per_site_dfs[0]
# tiktok_df = DataProcessing.concat_dfs(tiktok_dfs)

bluesky_dfs = hits_per_site_dfs[0]
bluesky_df = DataProcessing.concat_dfs(bluesky_dfs)

true_social_dfs = hits_per_site_dfs[1]
true_social_df = DataProcessing.concat_dfs(true_social_dfs)
true_social_df['text'] = true_social_df['content_cleaned']
true_social_df.tail(3)

,account,bookmarked,card,collected_by,content,content_cleaned,created_at,datatype,downvotes_count,emojis,...,reblog,editable,edited_at,version,application,text,tombstone,tv,Query Params,Site
7,"{'acct': 'Gutterman70', 'display_name': 'Gutterman70', 'id': '112994154379087050', 'username': 'Gutterman70'}",False,None,smat-scrapy-crawlers,"<p>C'mon NFL is a rigged thru sports betting bullshit sport, the Philadelphia Eagles should be crushing jerkoff Washington (Assholes) Redskins Oops Commanders, because Traitor Joe Duh Biden name changed after his crazy dog 🐕, the Eagles are allowing the commander's to score it's so obvious, fu NFL and like in 60s, 70s, 80s, 90s teams that where heading into playoffs played their 1st string players all the way to Super Bowl 🏈, after Kansas City, pulled there best players in playoffs first game, to avoid any injuries f that, I am also sick of bad refs, and all the bullshit celebration after scoring, overpaid jerkoffs, and stop the sports betting its obviously the issue, do it or lose fans, KABISH</p>","C'mon NFL is a rigged thru sports betting bullshit sport, the Philadelphia Eagles should be crushing jerkoff Washington (Assholes) Redskins Oops Commanders, because Traitor Joe Duh Biden name changed after his crazy dog 🐕, the Eagles are allowing the commander's to score it's so obvious, fu NFL and like in 60s, 70s, 80s, 90s teams that where heading into playoffs played their 1st string players all the way to Super Bowl 🏈, after Kansas City, pulled there best players in playoffs first game, to avoid any injuries f that, I am also sick of bad refs, and all the bullshit celebration after scoring, overpaid jerkoffs, and stop the sports betting its obviously the issue, do it or lose fans, KABISH",2025-01-26T22:36:52.972+00:00,post,0.0,[],...,NaN,False,NaN,1,NaN,"C'mon NFL is a rigged thru sports betting bullshit sport, the Philadelphia Eagles should be crushing jerkoff Washington (Assholes) Redskins Oops Commanders, because Traitor Joe Duh Biden name changed after his crazy dog 🐕, the Eagles are allowing the commander's to score it's so obvious, fu NFL and like in 60s, 70s, 80s, 90s teams that where heading into playoffs played their 1st string players all the way to Super Bowl 🏈, after Kansas City, pulled there best players in playoffs first game, to avoid any injuries f that, I am also sick of bad refs, and all the bullshit celebration after scoring, overpaid jerkoffs, and stop the sports betting its obviously the issue, do it or lose fans, KABISH",NaN,NaN,"{'term': '(NFL OR nfl) AND (playoffs) AND (Super Bowl LIX)', 'limit': 10, 'site': 'truth_social', 'since': '2025-01-25', 'until': '2025-02-09', 'querytype': 'query_string', 'model': 'user'}",truth_social
8,"{'acct': 'victoria_56', 'display_name': 'Victoria', 'id': '109875466344682540', 'username': 'victoria_56'}",False,None,smat-scrapy-crawlers,<p>I hear NFL playoffs are rigged and the two teams going to the Super Bowl will be the Eagles and Chiefs. I would like to see the Eagles and Bills</p>,I hear NFL playoffs are rigged and the two teams going to the Super Bowl will be the Eagles and Chiefs. I would like to see the Eagles and Bills,2025-01-26T23:41:34.560+00:00,post,0.0,[],...,NaN,False,NaN,1,NaN,I hear NFL playoffs are rigged and the two teams going to the Super Bowl will be the Eagles and Chiefs. I would like to see the Eagles and Bills,NaN,NaN,"{'term': '(NFL OR nfl) AND (playoffs) AND (Super Bowl LIX)', 'limit': 10, 'site': 'truth_social', 'since': '2025-01-25', 'until': '2025-02-09', 'querytype': 'query_string', 'model': 'user'}",truth_social
9,"{'acct': 'Artbyunclejunior', 'display_name': 'ArtbyUncleJR', 'id': '107834646100522511', 'username': 'Artbyunclejunior'}",False,"{'author_name': '', 'author_url': '', 'blurhash': 'UCCY?#9E^-D$xC-qaJxu9E?IE0-;XUE0kXIU', 'description': '', 'embed_url': '', 'height': 360, 'html': '<iframe src=""https://www.youtube.com/embed/5s0pxJLM_7M"" width=""640"" height=""360"" scrolling=""no"" frameborder=""0"" 

## Detect Prediction Label with LLM + Majority Vote

In [9]:
tgmf = TextGenerationModelFactory()

# Groq Cloud (https://console.groq.com/docs/overview)
gemma_29b_generation_model = tgmf.create_instance('gemma2-9b-it') 
llama_318b_instant_generation_model = tgmf.create_instance('llama-3.1-8b-instant') 
llama_3370b_versatile_generation_model = tgmf.create_instance('llama-3.3-70b-versatile')  
llama_guard_4_12b_generation_model = tgmf.create_instance('meta-llama/llama-guard-4-12b')  

models = [gemma_29b_generation_model, llama_318b_instant_generation_model, llama_3370b_versatile_generation_model, llama_guard_4_12b_generation_model]

# models = [gemma_29b_generation_model, llama_318b_instant_generation_model, llama_3370b_versatile_generation_model]
# models = [gemma_29b_generation_model, llama_318b_instant_generation_model, llama_guard_4_12b_generation_model]

In [10]:
def count_label(current_label, label_to_counts):
    """
    Given a sentence, use any LLM from text_generation_models to detect if it's a prediction or not.

    Parameters
    ----------
    current_label : pd.DataFrame
        Label from specific LLM on if it's a prediction or not
    label_to_counts : dict
        Data with the model_label : count

    Returns
    -------
    None
        Only update dictionary: label_to_counts
    
    
    """
    current_label = current_label.strip()
    # print(f"\tBase label_to_counts: {label_to_counts}")
    if current_label in label_to_counts.keys():
        old_count = label_to_counts[current_label]
        
        update_count = old_count + 1
        label_to_counts[current_label] = update_count
        # print(f"\tUpdated label_to_counts: {label_to_counts}")
    else:
        label_to_counts[current_label] = 1
        # print(f"\tUpdated label_to_counts: {label_to_counts}")

In [11]:
def get_majority_vote(data):
    """
    Given a dictionary, compute majority vote

    Parameters
    ----------
    data : dict
        Data with the model_label : count

    Returns
    -------
    str
        The label of the mode

    """
    mode = max(data.values()) 
    for key, val in data.items():
        if val == mode:
            if mode == 1: 
                majority_vote_label = 'None as all are different labels'
            else:
                majority_vote_label = key

    return majority_vote_label

In [ ]:
def detect_predictions_with_llms(df: pd.DataFrame, notebook_dir: str, site: str):
    """
    Given a sentence, use any LLM from text_generation_models to detect if it's a prediction or not.

    Parameters
    ----------
    df : pd.DataFrame
        Data with the sentences we want to label
    notebook_dir : str
        The location of this notebook, so we can save files using relative paths
    site : str
        Source of the data ('tiktok', 'bluesky', 'truth social', 'llm generated', etc).

    Returns
    -------
    pd.DataFrame
        The mappings of 1 (sentence) : many LLMs
        The mappings of 1 LLM : 1 prediction label
    
    
    """
    labels = []
    batch_size = 10
    show_data = 1

    sentences = DataProcessing.df_to_list(df, col='text')
    meta_data_df = df.drop(columns=['text'])

    
    for batch_idx in tqdm(range(0, len(sentences), batch_size)):
        batch = sentences[batch_idx:batch_idx+batch_size]
        for sentences_idx, sentence in enumerate(batch):
            label_to_count = {}
            # sentence = batch[sentences_idx]
            sent_meta_data_series = meta_data_df.iloc[batch_idx + sentences_idx]
            sent_meta_data = sent_meta_data_series.to_dict()
            if show_data <=3: 
                # print(f"\>>>Sentence: {sentence} --- {site}\nMetadata: {sent_meta_data}")
                # print(f"\n>>>Sentence: {sentence} --- {site}")
                print(f">>>Sentence: {sentence}")
                show_data += 1
                # break
            prompt = f"Given this sentence ({sentence}), state if the sentence is a prediction, not a prediction, or not enough information. Also, if it is a prediction, state that it is prediction along with the prediction domain if any are finance, health, weather, policy, sports, or miscellaneous. Do not explain or provide any other details. Remember, your responses are discrete corresponding to the states in the list; [prediction-finance, prediction-health, prediction-weather, prediction-policy, prediction-sports, prediction-miscellaneous, not a prediction, not enough information]."
            
            sentence_labels = []
            # print(f"Prompt: {prompt}")
            for model in models:  
                input_prompt = model.user(prompt)
                # print(input_prompt)  
                
                raw_text_llm_generation = model.chat_completion([input_prompt])
                # print(raw_text_llm_generation)
                # print("====================================")
                for line in raw_text_llm_generation.split("\n"):
                    # print(line)
                    if line.strip():
                        label = line.strip()
                        # print(f"\tModel -> Label: {model.__name__()} -> {label}")
                        count_label(label, label_to_count)

                        sentence_labels.append({
                            "sentence": sentence,
                            "model": model.__name__(),
                            "label": label,
                            "meta_data": sent_meta_data
                        })

            majority_vote_label = get_majority_vote(label_to_count)
            # print(f"\t\tMajority vote: {majority_vote_label}")

            # Attach majority vote to each model's label
            for entry in sentence_labels:
                entry["majority_vote"] = majority_vote_label
                labels.append(entry)

        print()
        save_dir = os.path.dirname(notebook_dir)
        # print(f"Site: {site}")
        save_dir = os.path.join(save_dir, 'data', 'open_measures', f"{site}s")
        # print(f"\n\tLabels: {labels}")
        DataProcessing.save_to_json(labels, save_dir, site)
    return labels

In [17]:
# tiktok_site = tiktok_df['Site'][0]
# llms_generated_for_tiktok_comments = detect_predictions_with_llms(tiktok_df, notebook_dir, tiktok_site)
# llms_generated_for_tiktok_comments_df = pd.DataFrame(llms_generated_for_tiktok_comments).rename(columns={'sentence': 'Text', 'model': 'Model', 'label': 'Label'})
# llms_generated_for_tiktok_comments_df['Site'] = tiktok_site
# llms_generated_for_tiktok_comments_df.head(7)

In [18]:
updated_column_names={'sentence': 'Text', 'model': 'Model', 'label': 'Label', 'meta_data': 'Meta Data', 'majority_vote': 'Majority Vote'}

In [19]:
bluesky_site = bluesky_df['Site'][0]
# bluesky_site = 'bluesky'
llms_generated_for_bluesocial_comments = detect_predictions_with_llms(bluesky_df, notebook_dir, bluesky_site)
llms_generated_for_bluesocial_comments_df = pd.DataFrame(llms_generated_for_bluesocial_comments).rename(columns=updated_column_names)
llms_generated_for_bluesocial_comments_df.head(7)

  0%|          | 0/1 [00:00<?, ?it/s]

>>>Sentence: Column: is this the year Josh Allen and the Bills slay the dragon in KC and advance to the Super Bowl? My picks for the AFC-NFC conference championship games  

www.scoopswithdannymac.com/bernies-nfl-...
>>>Sentence: Eagles vs. Commanders odds, line, spread: 2025 NFC Championship Game predictions by model on 31-15 roll​on January 24, 2025 at 10:31 pm

SportsLine's model simulated Sunday's Washington vs. Philadelphia NFC Championship Game 10,000 times and revealed its NFL best bets for the 2025 NFL playoffs…
>>>Sentence: Alex Marlow: Follow the Money—NFL Rigging Playoffs for the Chiefs So Taylor Swift Will Attend Super Bowl


100%|██████████| 1/1 [00:12<00:00, 12.40s/it]


	Saved to /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/data/open_measures/blueskys/bluesky-60.json


,Text,Model,Label,Meta Data,Majority Vote
0,Column: is this the year Josh Allen and the Bills slay the dragon in KC and advance to the Super Bowl? My picks for the AFC-NFC conference championship games \n\nwww.scoopswithdannymac.com/bernies-nfl-...,gemma2-9b-it,prediction-sports,"{'$type': 'app.bsky.feed.post', 'author': 'did:plc:zialmxrbi3qx7fpogrov6qwf', 'authorProfile': {'did': 'did:plc:zialmxrbi3qx7fpogrov6qwf', 'handle': 'bmiklasz.bsky.social'}, 'cid': 'bafyreihigyfstdjmhurzc2qwblypbuff6rnwc5vlnuvrqggx7qfcmhyf6m', 'createdAt': '2025-01-25T00:45:29.712Z', 'embed': {'$type': 'app.bsky.embed.external', 'external': {'$type': 'app.bsky.embed.external#external', 'description': 'Hello, again. I am looking forward to Sunday. I’ll have some chili on the stove, cold beverages in the fridge, and the TV set will be ready to go when it’s time to snap the football. It’s AFC-NFC Cham...', 'thumb': {'$type': 'blob', 'mime_type': 'image/jpeg', 'ref_str': 'bafkreif2qqdwqpprxyzca24tu4vpurbmord76rlqbzulklbrkb3346hl5i', 'size': 627377}, 'title': 'Bernie's NFL Conference Champi...",prediction-sports
1,Column: is this the year Josh Allen and the Bills slay the dragon in KC and advance to the Super Bowl? My picks for the AFC-NFC conference championship games \n\nwww.scoopswithdannymac.com/bernies-nfl-...,llama-3.1-8b-instant,prediction-sports,"{'$type': 'app.bsky.feed.post', 'author': 'did:plc:zialmxrbi3qx7fpogrov6qwf', 'authorProfile': {'did': 'did:plc:zialmxrbi3qx7fpogrov6qwf', 'handle': 'bmiklasz.bsky.social'}, 'cid': 'bafyreihigyfstdjmhurzc2qwblypbuff6rnwc5vlnuvrqggx7qfcmhyf6m', 'createdAt': '2025-01-25T00:45:29.712Z', 'embed': {'$type': 'app.bsky.embed.external', 'external': {'$type': 'app.bsky.embed.external#external', 'description': 'Hello, again. I am looking forward to Sunday. I’ll have some chili on the stove, cold beverages in the fridge, and the TV set will be ready to go when it’s time to snap the football. It’s AFC-NFC Cham...', 'thumb': {'$type': 'blob', 'mime_type': 'image/jpeg', 'ref_str': 'bafkreif2qqdwqpprxyzca24tu4vpurbmord76rlqbzulklbrkb3346hl5i', 'size': 627377}, 'title': 'Bernie's NFL Conference Champi...",prediction-sports
2,Column: is this the year Josh Allen and the Bills slay the dragon in KC and advance to the Super Bowl? My picks for the AFC-NFC conference championship games \n\nwww.scoopswithdannymac.com/bernies-nfl-...,llama-3.3-70b-versatile,prediction-sports,"{'$type': 'app.bsky.feed.post', 'author': 'did:plc:zialmxrbi3qx7fpogrov6qwf', 'authorProfile': {'did': 'did:plc:zialmxrbi3qx7fpogrov6qwf', 'handle': 'bmiklasz.bsky.social'}, 'cid': 'bafyreihigyfstdjmhurzc2qwblypbuff6rnwc5vlnuvrqggx7qfcmhyf6m', 'createdAt': '2025-01-25T00:45:29.712Z', 'embed': {'$type': 'app.bsky.embed.external', 'external': {'$type': 'app.bsky.embed.external#external', 'description': 'Hello, again. I am looking forward to Sunday. I’ll have some chili on the stove, cold beverages in the fridge, and the TV set will be ready to go when it’s time to snap the football. It’s AFC-NFC Cham...', 'thumb': {'$type': 'blob', 'mime_type': 'image/jpeg', 'ref_str': 'bafkreif2qqdwqpprxyzca24tu4vpurbmord76rlqbzulklbrkb3346hl5i', 'size': 627377}, 'title': 'Bernie's NFL Conference Champi...",prediction-sports
3,Column: is this the year Josh Allen and the Bills slay the dragon in KC and advance to the Super Bowl? My picks for the AFC-NFC conference championship games \n\nwww.scoopswithdannymac.com/bernies-nfl-...,meta-llama/llama-guard-4-12b,safe,"{'$type': 'app.bsky.feed.post', 'author': 'did:plc:zialmxrbi3qx7fpogrov6qwf', 'authorProfile': {'did': 'did:plc:zialmxrbi3qx7fpogrov6qwf', 'handle': 'bmiklasz.bsky.social'}, 'cid': 'bafyreihigyfstdjmhurzc2qwblypbuff6rnwc5vlnuvrqggx7qfcmhyf6m', 'createdAt': '2025-01-25T00:45:29.712Z', 'embed': {'$type': 'app.bsky.embed.external', 'external': {'$type': 'app.bsky.embed.external#external', 'description': 'Hello, again. I am looking forward to Sunday. I’ll have some chili on the stove, cold beverages in the fridge, and the

In [20]:
true_social_site = true_social_df['Site'][0]
# true_social_site = 'Site'
llms_generated_for_true_social_comments = detect_predictions_with_llms(true_social_df, notebook_dir, true_social_site)
llms_generated_for_true_social_comments_df = pd.DataFrame(llms_generated_for_true_social_comments).rename(columns=updated_column_names)
llms_generated_for_true_social_comments_df.head(7)

  0%|          | 0/1 [00:00<?, ?it/s]

>>>Sentence: Alex Marlow: Follow the Money—NFL Rigging Playoffs for the Chiefs So Taylor Swift Will Attend Super Bowl https://www.breitbart.com/clips/2025/01/24/alex-marlow-follow-the-money-nfl-rigging-playoffs-for-the-chiefs-so-taylor-swift-will-attend-super-bowl/
>>>Sentence: EXACTLY! WE'VE ALL THOUGHT THIS Alex Marlow: Follow the Money—NFL Rigging #playoffs for the #Chiefs So #TaylorSwift Will Attend Super Bowl via @breitbartnewshttps://www.breitbart.com/clips/2025/01/24/alex-marlow-follow-the-money-nfl-rigging-playoffs-for-the-chiefs-so-taylor-swift-will-attend-super-bowl/
>>>Sentence: Alex Marlow: Follow the Money—NFL Rigging Playoffs for the Chiefs So Taylor Swift Will Attend Super Bowlhttps://www.breitbart.com/clips/2025/01/24/alex-marlow-follow-the-money-nfl-rigging-playoffs-for-the-chiefs-so-taylor-swift-will-attend-super-bowl/


100%|██████████| 1/1 [00:11<00:00, 11.88s/it]


	Saved to /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/data/open_measures/truth_socials/truth_social-21.json


,Text,Model,Label,Meta Data,Majority Vote
0,Alex Marlow: Follow the Money—NFL Rigging Playoffs for the Chiefs So Taylor Swift Will Attend Super Bowl https://www.breitbart.com/clips/2025/01/24/alex-marlow-follow-the-money-nfl-rigging-playoffs-for-the-chiefs-so-taylor-swift-will-attend-super-bowl/,gemma2-9b-it,prediction-sports,"{'account': {'acct': 'PowerDownMedia', 'display_name': 'ᑭᗝᗯᗴᖇ ᗪᗝᗯᑎ ᗰᗴᗪᎥᗩ', 'id': '108323694466303680', 'username': 'PowerDownMedia'}, 'bookmarked': False, 'card': {'author_name': '', 'author_url': '', 'blurhash': 'UCIz;r~T7i%#yYD%%3oJE{nNrWxBDiNKMwR+', 'description': 'On Friday's ""Alex Marlow Show,"" host and Breitbart Editor-in-Chief Alex Marlow talked about the NFL playoffs. Marlow said, ""You guys know the NFL just | Clips', 'embed_url': '', 'group': None, 'height': 335, 'html': '', 'id': None, 'image': 'https://static-assets-1.truthsocial.com/tmtg:prime-ts-assets/cache/preview_cards/images/047/982/221/original/ce0312cfd1c0bb34.jpg', 'links': None, 'provider_name': 'www.breitbart.com', 'provider_url': '', 'title': 'Alex Marlow: Follow the Money—NFL Rigging Playoffs for the Chiefs So T...",prediction-sports
1,Alex Marlow: Follow the Money—NFL Rigging Playoffs for the Chiefs So Taylor Swift Will Attend Super Bowl https://www.breitbart.com/clips/2025/01/24/alex-marlow-follow-the-money-nfl-rigging-playoffs-for-the-chiefs-so-taylor-swift-will-attend-super-bowl/,llama-3.1-8b-instant,prediction-sports,"{'account': {'acct': 'PowerDownMedia', 'display_name': 'ᑭᗝᗯᗴᖇ ᗪᗝᗯᑎ ᗰᗴᗪᎥᗩ', 'id': '108323694466303680', 'username': 'PowerDownMedia'}, 'bookmarked': False, 'card': {'author_name': '', 'author_url': '', 'blurhash': 'UCIz;r~T7i%#yYD%%3oJE{nNrWxBDiNKMwR+', 'description': 'On Friday's ""Alex Marlow Show,"" host and Breitbart Editor-in-Chief Alex Marlow talked about the NFL playoffs. Marlow said, ""You guys know the NFL just | Clips', 'embed_url': '', 'group': None, 'height': 335, 'html': '', 'id': None, 'image': 'https://static-assets-1.truthsocial.com/tmtg:prime-ts-assets/cache/preview_cards/images/047/982/221/original/ce0312cfd1c0bb34.jpg', 'links': None, 'provider_name': 'www.breitbart.com', 'provider_url': '', 'title': 'Alex Marlow: Follow the Money—NFL Rigging Playoffs for the Chiefs So T...",prediction-sports
2,Alex Marlow: Follow the Money—NFL Rigging Playoffs for the Chiefs So Taylor Swift Will Attend Super Bowl https://www.breitbart.com/clips/2025/01/24/alex-marlow-follow-the-money-nfl-rigging-playoffs-for-the-chiefs-so-taylor-swift-will-attend-super-bowl/,llama-3.3-70b-versatile,prediction-sports,"{'account': {'acct': 'PowerDownMedia', 'display_name': 'ᑭᗝᗯᗴᖇ ᗪᗝᗯᑎ ᗰᗴᗪᎥᗩ', 'id': '108323694466303680', 'username': 'PowerDownMedia'}, 'bookmarked': False, 'card': {'author_name': '', 'author_url': '', 'blurhash': 'UCIz;r~T7i%#yYD%%3oJE{nNrWxBDiNKMwR+', 'description': 'On Friday's ""Alex Marlow Show,"" host and Breitbart Editor-in-Chief Alex Marlow talked about the NFL playoffs. Marlow said, ""You guys know the NFL just | Clips', 'embed_url': '', 'group': None, 'height': 335, 'html': '', 'id': None, 'image': 'https://static-assets-1.truthsocial.com/tmtg:prime-ts-assets/cache/preview_cards/images/047/982/221/original/ce0312cfd1c0bb34.jpg', 'links': None, 'provider_name': 'www.breitbart.com', 'provider_url': '', 'title': 'Alex Marlow: Follow the Money—NFL Rigging Playoffs for the Chiefs So T...",prediction-sports
3,Alex Marlow: Follow the Money—NFL Rigging Playoffs for the Chiefs So Taylor Swift Will Attend Super Bowl https://www.breitbart.com/clips/2025/01/24/alex-marlow-follow-the-money-nfl-rigging-playoffs-for-the-chiefs-so-taylor-swift-will-attend-super-bowl/,meta-llama/llama-guard-4-12b,unsafe,"{'account': {'acct': 'PowerDownMedia', 'display_name': 'ᑭᗝᗯᗴᖇ ᗪᗝᗯᑎ ᗰᗴᗪᎥᗩ', 'id': '108323694466303680', 'username': 'PowerDownMedia'}, 'bookmarked': False, 'card': {'author_name': '', 'author_url': '', 'blurhash': 'UCIz;r~T7i%#yYD%%3oJE{nNrWxBDiNKMwR+', 'description': 'On Friday's ""Alex Marlow Show,"" host and Breitbar

In [21]:
llms_generated_for_true_social_comments_df

,Text,Model,Label,Meta Data,Majority Vote
0,Alex Marlow: Follow the Money—NFL Rigging Playoffs for the Chiefs So Taylor Swift Will Attend Super Bowl https://www.breitbart.com/clips/2025/01/24/alex-marlow-follow-the-money-nfl-rigging-playoffs-for-the-chiefs-so-taylor-swift-will-attend-super-bowl/,gemma2-9b-it,prediction-sports,"{'account': {'acct': 'PowerDownMedia', 'display_name': 'ᑭᗝᗯᗴᖇ ᗪᗝᗯᑎ ᗰᗴᗪᎥᗩ', 'id': '108323694466303680', 'username': 'PowerDownMedia'}, 'bookmarked': False, 'card': {'author_name': '', 'author_url': '', 'blurhash': 'UCIz;r~T7i%#yYD%%3oJE{nNrWxBDiNKMwR+', 'description': 'On Friday's ""Alex Marlow Show,"" host and Breitbart Editor-in-Chief Alex Marlow talked about the NFL playoffs. Marlow said, ""You guys know the NFL just | Clips', 'embed_url': '', 'group': None, 'height': 335, 'html': '', 'id': None, 'image': 'https://static-assets-1.truthsocial.com/tmtg:prime-ts-assets/cache/preview_cards/images/047/982/221/original/ce0312cfd1c0bb34.jpg', 'links': None, 'provider_name': 'www.breitbart.com', 'provider_url': '', 'title': 'Alex Marlow: Follow the Money—NFL Rigging Playoffs for the Chiefs So T...",prediction-sports
1,Alex Marlow: Follow the Money—NFL Rigging Playoffs for the Chiefs So Taylor Swift Will Attend Super Bowl https://www.breitbart.com/clips/2025/01/24/alex-marlow-follow-the-money-nfl-rigging-playoffs-for-the-chiefs-so-taylor-swift-will-attend-super-bowl/,llama-3.1-8b-instant,prediction-sports,"{'account': {'acct': 'PowerDownMedia', 'display_name': 'ᑭᗝᗯᗴᖇ ᗪᗝᗯᑎ ᗰᗴᗪᎥᗩ', 'id': '108323694466303680', 'username': 'PowerDownMedia'}, 'bookmarked': False, 'card': {'author_name': '', 'author_url': '', 'blurhash': 'UCIz;r~T7i%#yYD%%3oJE{nNrWxBDiNKMwR+', 'description': 'On Friday's ""Alex Marlow Show,"" host and Breitbart Editor-in-Chief Alex Marlow talked about the NFL playoffs. Marlow said, ""You guys know the NFL just | Clips', 'embed_url': '', 'group': None, 'height': 335, 'html': '', 'id': None, 'image': 'https://static-assets-1.truthsocial.com/tmtg:prime-ts-assets/cache/preview_cards/images/047/982/221/original/ce0312cfd1c0bb34.jpg', 'links': None, 'provider_name': 'www.breitbart.com', 'provider_url': '', 'title': 'Alex Marlow: Follow the Money—NFL Rigging Playoffs for the Chiefs So T...",prediction-sports
2,Alex Marlow: Follow the Money—NFL Rigging Playoffs for the Chiefs So Taylor Swift Will Attend Super Bowl https://www.breitbart.com/clips/2025/01/24/alex-marlow-follow-the-money-nfl-rigging-playoffs-for-the-chiefs-so-taylor-swift-will-attend-super-bowl/,llama-3.3-70b-versatile,prediction-sports,"{'account': {'acct': 'PowerDownMedia', 'display_name': 'ᑭᗝᗯᗴᖇ ᗪᗝᗯᑎ ᗰᗴᗪᎥᗩ', 'id': '108323694466303680', 'username': 'PowerDownMedia'}, 'bookmarked': False, 'card': {'author_name': '', 'author_url': '', 'blurhash': 'UCIz;r~T7i%#yYD%%3oJE{nNrWxBDiNKMwR+', 'description': 'On Friday's ""Alex Marlow Show,"" host and Breitbart Editor-in-Chief Alex Marlow talked about the NFL playoffs. Marlow said, ""You guys know the NFL just | Clips', 'embed_url': '', 'group': None, 'height': 335, 'html': '', 'id': None, 'image': 'https://static-assets-1.truthsocial.com/tmtg:prime-ts-assets/cache/preview_cards/images/047/982/221/original/ce0312cfd1c0bb34.jpg', 'links': None, 'provider_name': 'www.breitbart.com', 'provider_url': '', 'title': 'Alex Marlow: Follow the Money—NFL Rigging Playoffs for the Chiefs So T...",prediction-sports
3,Alex Marlow: Follow the Money—NFL Rigging Playoffs for the Chiefs So Taylor Swift Will Attend Super Bowl https://www.breitbart.com/clips/2025/01/24/alex-marlow-follow-the-money-nfl-rigging-playoffs-for-the-chiefs-so-taylor-swift-will-attend-super-bowl/,meta-llama/llama-guard-4-12b,unsafe,"{'account': {'acct': 'PowerDownMedia', 'display_name': 'ᑭᗝᗯᗴᖇ ᗪᗝᗯᑎ ᗰᗴᗪᎥᗩ', 'id': '108323694466303680', 'username': 'PowerDownMedia'}, 'bookmarked': False, 'card': {'author_name': '', 'author_url': '', 'blurhash': 'UCIz;r~T7i%#yYD%%3oJE{nNrWxBDiNKMwR+', 'description': 'On Friday's ""Alex Marlow Show,"" host and Breitbar